In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

MINUTES_TO_HOURS = 1/60

# Bid Offer Analysis
This analysis focuses on March 19th 2022, based upon the analysis done in the other notebooks which identified this as a period of negative prices.

It focuses upon accepted offers to reduce generation.

In [2]:
df_bm_units = pd.read_excel("../data/BMUFuelType.xls", header=0)

In [3]:
df_fpn = pd.read_csv(
    "../data/PhysicalData_20220319_27.csv",
    skiprows=lambda x: x > 1541 or x == 0,
    names=[
        "Notification Type",
        "Unit ID",
        "Settlement Period",
        "From Time",
        "From Level",
        "To Time",
        "To Level",
    ],
    parse_dates=["From Time", "To Time"],
    date_parser=lambda x: pd.to_datetime(x, format="%Y%m%d%H%M%S"),
)  # This includes other kinds of data with different cols

df_fpn = (
    df_fpn.set_index("Unit ID")
    .join(df_bm_units.set_index("SETT_BMU_ID")["FUEL TYPE"])
    .rename(columns={"FUEL TYPE": "Fuel Type"})
)

There are many different forms here:
- FPN (Final Physical Notification): what the operators tell the grid they're going to generate
- QPN (Quiescent Physical Notifications)
- MEL (Maximum Export Limit)
- MIL (Maximum Import Limit)
- BOAL (Bid-Offer Acceptance Level Data)

In [4]:
df_boal = pd.read_csv(
    "../data/PhysicalData_20220319_27.csv",
    skiprows=lambda x: x < 5718,
    names=[
        "Notification Type",
        "Unit ID",
        "Accept ID",
        "Accept Time",
        "Deemed Flag",
        "SO Flag",
        "STOR Provider Flag",
        "RR Instruction Flag",
        "RR Schedule Flag",
        "From Time",
        "From Level",
        "To Time",
        "To Level",
    ],
    parse_dates=["Accept Time", "From Time", "To Time"],
    date_parser=lambda x: pd.to_datetime(x, format="%Y%m%d%H%M%S"),
)  # This includes other kinds of data with different cols

df_boal = df_boal[df_boal["Notification Type"] == "BOALF"]
df_boal['Accept ID'] = df_boal['Accept ID'].astype(int)
df_boal = (
    df_boal.set_index("Unit ID")
    .join(df_bm_units.set_index("SETT_BMU_ID")["FUEL TYPE"])
    .rename(columns={"FUEL TYPE": "Fuel Type"})
)

In [5]:
BOOL_COLS = [
    "Deemed Flag",
    "SO Flag",
    "STOR Provider Flag",
    "RR Instruction Flag",
    "RR Schedule Flag",
]

df_boal[BOOL_COLS] = df_boal[BOOL_COLS].apply(lambda x: x == "T")

I believe the SO Flag is identifying System Operator actions i.e. the ones we are interested in

In [6]:
df_boal[df_boal["SO Flag"]]

,Notification Type,Accept ID,Accept Time,Deemed Flag,SO Flag,STOR Provider Flag,RR Instruction Flag,RR Schedule Flag,From Time,From Level,To Time,To Level,Fuel Type
T_CARR-1,BOALF,79613,2022-03-19 12:09:00,False,True,False,False,False,2022-03-19 13:00:00,176.0,2022-03-19 13:30:00,176.0,CCGT
T_CARR-1,BOALF,79614,2022-03-19 12:44:00,False,True,False,False,False,2022-03-19 12:46:00,176.0,2022-03-19 13:46:00,176.0,CCGT
T_CARR-1,BOALF,79615,2022-03-19 13:26:00,False,True,False,False,False,2022-03-19 13:28:00,176.0,2022-03-19 14:13:00,176.0,CCGT
T_EAAO-1,BOALF,4379,2022-03-19 12:46:00,False,True,False,False,False,2022-03-19 12:49:00,230.0,2022-03-19 13:19:00,230.0,WIND
T_EAAO-1,BOALF,4379,2022-03-19 12:46:00,False,True,False,False,False,2022-03-19 13:19:00,230.0,2022-03-19 13:22:00,336.0,WIND
T_EAAO-1,BOALF,4380,2022-03-19 13:04:00,False,True,False,False,False,2022-03-19 13:19:00,230.0,2022-03-19 13:49:00,230.0,WIND
T_LAGA-1,BOALF,133192,2022-03-19 12:00:00,False,True,False,False,False,2022-03-19 13:20:00,0.0,2022-03-19 13:30:00,17.0,CCGT
T_LAGA-1,BOALF,133193,2022-03-19 12:32:00,False,True,False,False,False,2022-03-19 13:29:00,16.0,2022-03-19 14:00:00,71.0,CCGT
T_MRWD-1,BOALF,144962,2022-03-19 12:01:00,False,True,False,False,False,2022-03-19 12:59:00,450.0,2022-03-19 13:30:00,450.0,CCGT
T_MRWD-1,BOALF,144965,2022-03-19 12:32:00,False,True,False,False,False,2022-03-19 13:29:00,450.0,2022-03-19 14:00:00,450.0,CCGT


In [7]:
df_boal_wind = df_boal[df_boal["Fuel Type"] == "WIND"]

In [8]:
df_boal_wind

,Notification Type,Accept ID,Accept Time,Deemed Flag,SO Flag,STOR Provider Flag,RR Instruction Flag,RR Schedule Flag,From Time,From Level,To Time,To Level,Fuel Type
E_BTUIW-3,BOALF,3642,2022-03-19 12:35:00,False,False,False,False,False,2022-03-19 13:20:00,0.0,2022-03-19 13:22:00,43.0,WIND
E_BTUIW-3,BOALF,3643,2022-03-19 13:05:00,False,False,False,False,False,2022-03-19 13:20:00,0.0,2022-03-19 13:50:00,0.0,WIND
T_AKGLW-3,BOALF,2005,2022-03-19 12:26:00,False,False,False,False,False,2022-03-19 12:58:00,0.0,2022-03-19 13:03:00,48.0,WIND
T_AKGLW-3,BOALF,2006,2022-03-19 12:50:00,False,False,False,False,False,2022-03-19 12:52:00,0.0,2022-03-19 13:52:00,0.0,WIND
T_AKGLW-3,BOALF,2007,2022-03-19 13:02:00,False,False,False,False,False,2022-03-19 13:04:00,0.0,2022-03-19 14:19:00,0.0,WIND
...,...,...,...,...,...,...,...,...,...,...,...,...,...
T_WLNYO-3,BOALF,22300,2022-03-19 13:07:00,False,False,False,False,False,2022-03-19 13:17:00,0.0,2022-03-19 13:37:00,0.0,WIND
T_WLNYO-4,BOALF,23054,2022-03-19 12:30:00,False,False,False,False,False,2022-03-19 13:00:00,0.0,2022-03-19 13:06:00,253.0,WIND
T_WLNYO-4,BOALF,23055,2022-03-19 12:50:00,False,False,False,False,False,2022-03-19 13:00:00,0.0,2022-03-19 13:20:00,0.0,WIND
T_WLNYO-4,BOALF,23055,2022-03-19 12:50:00,False,False,False,False,False,2022-03-19 13:20:00,0.0,2022-03-19 13:26:00,262.0,WIND


# Identifying curtailment over all units

Curtailment is if the BOALF level is less than the FPN

i.e. somebody is being paid to turn it down!

In [9]:
# These are some helpers functions

def resolve_applied_bid_offer_level(df_linear: pd.DataFrame):
    """
    We can have multiple levels for a given timepoint, because levels are fixed
    at one point and then overwitten at a later timepoint, before the moment in
    question has arrived.
    
    We need to resolve them, choosing the latest possible commitment for each timepoint.
    
    We need to upsample the data first to achieve this.
    """
    
    out = []
    for accept_id, data in df_linear.groupby("Accept ID"):
        high_freq = (
            data.reset_index()
            .rename(columns={"index": "Unit"})
            .set_index("Time")
            .resample("T")
            .first()

        )
        out.append(high_freq.interpolate("ffill").fillna(method='ffill'))

    recombined = pd.concat(out)
    
    # Select the latest commitment for every timepoint
    resolved = recombined.reset_index().groupby('Time').last()
    
    return resolved

def linearize_physical_data(df: pd.DataFrame):
    """Convert a From/To horizontal format to a long format with values at different timepoitns"""

    from_columns = ["From Level", "From Time"]
    to_columns = ["To Level", "To Time"]

    base_columns = [x for x in df.columns.copy() if x not in from_columns + to_columns]

    df = pd.concat(
        (
            df[base_columns + from_columns].rename(
                columns={"From Level": "Level", "From Time": "Time"}
            ),
            df[base_columns + to_columns].rename(
                columns={"To Level": "Level", "To Time": "Time"}
            ),
        )
    )

    df['Level'] = df['Level'].astype(float)
    return df

def calculate_curtailment_in_mwh(unit_fpn_resolved, unit_boal_resolved) -> float:
    """
    Calculate the curtailment implied by the difference between FPN levels and BOAL
    
    """
    df_merged = unit_boal_resolved.join(unit_fpn_resolved, rsuffix='_FPN') 
    df_merged['delta'] = df_merged['Level_FPN'] - df_merged['Level']
    
    mw_minutes =  df_merged['delta'].sum()
    
    return mw_minutes * MINUTES_TO_HOURS

def calculate_generation_in_mwh(unit_fpn_resolved) -> float:
    """
    Calculate the total generation implied by the FPN levels
    
    """
    
    mw_minutes =  unit_fpn_resolved['Level'].sum()
    
    return mw_minutes * MINUTES_TO_HOURS



In [10]:
# now lets loop over all wind units

wind_units = df_boal_wind.index.unique()

print(wind_units)

total_curtailment = 0
total_generation = 0 
combined = []
for unit in wind_units:

    # get boas for that one wind unit
    df_boal_wind.loc[unit]
    
    # Make time linear
    df_boal_linear = linearize_physical_data(df_boal.loc[unit])
    df_boal_linear["Accept Time str"] = df_boal_linear["Accept Time"].astype(str)

    px.line(df_boal_linear, "Time", "Level", color="Accept Time str")
    
    # resolve boa data
    unit_boal_resolved = resolve_applied_bid_offer_level(df_boal_linear)
    unit_boal_resolved.head()
    
    # resolve fpn data
    df_fpn_one_unit = df_fpn.loc[unit]
    if type(df_fpn_one_unit) == pd.Series:
        df_fpn_one_unit = pd.DataFrame(df_fpn_one_unit).T

    unit_fpn_resolved = linearize_physical_data(df_fpn_one_unit).set_index('Time').resample('T').mean().interpolate()
    unit_fpn_resolved["Notification Type"] = 'FPN'
    unit_fpn_resolved["unit"] = unit
    
    # cmobind both BOA and FPN data
    combined_one_unit = pd.concat((unit_boal_resolved, unit_fpn_resolved))
    
    curtailment_in_mwh = calculate_curtailment_in_mwh(unit_fpn_resolved=unit_fpn_resolved, unit_boal_resolved=unit_boal_resolved)
    generation_in_mwh = calculate_generation_in_mwh(unit_fpn_resolved)
    print(f'Curtailment for {unit} is {curtailment_in_mwh:.2f} MWh. Generation was {generation_in_mwh:.2f} MWh')
    
#     px.line(combined_one_unit, x=combined_one_unit.index, y='Level', color='Notification Type')
    combined.append(combined_one_unit)
    total_curtailment += curtailment_in_mwh
    total_generation += generation_in_mwh

print(f'Total curtailment is {total_curtailment:.2f} MWh. The total generation was {total_generation:.2f} MWh.')
    
    
    
    
    

Index(['E_BTUIW-3', 'T_AKGLW-3', 'T_BDCHW-1', 'T_BEATO-3', 'T_BEATO-4',
       'T_BRBEO-1', 'T_DOREW-1', 'T_DOREW-2', 'T_EAAO-1', 'T_EAAO-2',
       'T_GORDW-2', 'T_GRIFW-1', 'T_GRIFW-2', 'T_HOWAO-1', 'T_HOWAO-2',
       'T_HOWAO-3', 'T_KPMRW-1', 'T_MIDMW-1', 'T_MOWEO-1', 'T_MOWEO-2',
       'T_MOWEO-3', 'T_PNYCW-1', 'T_STRNW-1', 'T_WLNYO-3', 'T_WLNYO-4'],
      dtype='object')
Curtailment for E_BTUIW-3 is 7.88 MWh. Generation was 22.22 MWh
Curtailment for T_AKGLW-3 is 25.06 MWh. Generation was 25.06 MWh
Curtailment for T_BDCHW-1 is 7.23 MWh. Generation was 7.23 MWh
Curtailment for T_BEATO-3 is 68.88 MWh. Generation was 68.88 MWh
Curtailment for T_BEATO-4 is 78.07 MWh. Generation was 78.07 MWh
Curtailment for T_BRBEO-1 is 69.23 MWh. Generation was 107.98 MWh
Curtailment for T_DOREW-1 is 30.74 MWh. Generation was 30.74 MWh
Curtailment for T_DOREW-2 is 24.54 MWh. Generation was 26.09 MWh
Curtailment for T_EAAO-1 is 56.27 MWh. Generation was 173.60 MWh
Curtailment for T_EAAO-2 is 13.53 MW

In [11]:
# lets plot it


traces = []
for combined_one_unit in combined:
    
    unit = combined_one_unit['unit'].iloc[-1]
    colour = 
    
    for notification_type in ['BOALF','FPN']:
        idx = combined_one_unit['Notification Type'] == notification_type
        x = combined_one_unit[idx].index
        y = combined_one_unit[idx]['Level']
                             
        trace = go.Scatter(x=x, y=y, name=f'{unit} - {notification_type}',)
        traces.append(trace)
    
fig = go.Figure(data=traces)

fig.show()

SyntaxError: invalid syntax (3273124099.py, line 8)

# TODO

- run over on day
- run over 1 year